## Preprocessing

In [1]:
import os
import pickle
import pandas as pd
import numpy as np
from scipy.signal import argrelextrema
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

In [10]:
def find_svc_files(root_dir):
    svc_files = []
    
    # Walk through all directories and subdirectories
    for dirpath, _, filenames in os.walk(root_dir):
        for file in filenames:
            if file.endswith("_1_1.svc"):  # Check if the file has .svc extension
                svc_files.append(os.path.join(dirpath, file))  # Store full path
    
    return svc_files

def feature_extraction_HandMotion(folder_path, label):
    txt_files = [f for f in os.listdir(folder_path) if f.endswith(".txt")]
    dataframes = []

    for txt_file in txt_files:
        txt_path = os.path.join(folder_path, txt_file)
        
        df = pd.read_csv(txt_path, delimiter=";", names=["X", "Y", "Z", "Pressure", "GripAngle", "Timestamp", "Test ID"])
        df = df[df["Pressure"] != 0]
        # Compute delta time
        df["dt"] = df["Timestamp"].diff()
        df["dt"] = df["dt"].replace(0, np.nan).interpolate().ffill().bfill()  # No zeros!
          # Avoid division by zero
        # Compute displacement
        df["dx"] = df["X"].diff().fillna(0)
        df["dy"] = df["Y"].diff().fillna(0)
        # Compute trajectory length
        df["trajectory_length"] = np.sqrt(df["dx"]**2 + df["dy"]**2)
        # Stroke speed
        stroke_speed = df["trajectory_length"].sum() / (df["Timestamp"].iloc[-1] - df["Timestamp"].iloc[0])
        # Compute velocity
        df["velocity"] = df["trajectory_length"] / df["dt"]
        df["velocity"] = df["velocity"].replace([np.inf, -np.inf], np.nan).interpolate().ffill().bfill()
        # Compute acceleration
        df["acceleration"] = df["velocity"].diff().fillna(0) / df["dt"]
        df["acceleration"] = df["acceleration"].replace([np.inf, -np.inf], np.nan).interpolate().ffill().bfill()
        # Compute jerk
        df["jerk"] = df["acceleration"].diff().fillna(0) / df["dt"]
        df["jerk"] = df["jerk"].replace([np.inf, -np.inf], np.nan).interpolate().ffill().bfill()
        # Number of changes in velocity and acceleration direction
        ncv = len(argrelextrema(df["velocity"].values, np.less)[0]) + len(argrelextrema(df["velocity"].values, np.greater)[0])
        nca = len(argrelextrema(df["acceleration"].values, np.less)[0]) + len(argrelextrema(df["acceleration"].values, np.greater)[0])
        # Path efficiency
        euclidean_distance = np.sqrt((df["X"].iloc[-1] - df["X"].iloc[0])**2 + (df["Y"].iloc[-1] - df["Y"].iloc[0])**2)
        path_efficiency = euclidean_distance / df["trajectory_length"].sum()
        # Stroke duration
        stroke_duration = df["Timestamp"].iloc[-1] - df["Timestamp"].iloc[0]
        # Pressure features
        mean_pressure = df["Pressure"].mean()
        std_pressure = df["Pressure"].std()
        num_pressure_drops = sum(df["Pressure"].diff().fillna(0) < -0.1)

        # Feature DataFrame
        feature_data = pd.DataFrame([[
            stroke_speed, df["velocity"].mean(), df["acceleration"].mean(), df["jerk"].mean(), 
            ncv, nca, path_efficiency, stroke_duration, mean_pressure, std_pressure, num_pressure_drops, label
        ]], columns=[
            "stroke_speed", "mean_velocity", "mean_acceleration", "mean_jerk", 
            "NCV", "NCA", "path_efficiency", "stroke_duration",
            "mean_pressure", "std_pressure", "num_pressure_drops", "label"
        ])

        dataframes.append(feature_data)

    # Combine all DataFrames
    if dataframes:
        return pd.concat(dataframes, ignore_index=True)
    else:
        return pd.DataFrame()


columns = ["Y", "X", "Timestamp", "button state", "azimuth", "altitude", "Pressure"]
dataframes_PaHaW = []


def feature_extraction_HandMotion_ext(folder_path, label):
    
    svc_files_list = find_svc_files(folder_path)

    for file_path in svc_files_list:
        df = pd.read_csv(file_path, delimiter=" ", header=None, names=columns, skiprows=1)

        df = df[df["Pressure"] != 0]
        # Compute delta time
        df["dt"] = df["Timestamp"].diff()
        df["dt"] = df["dt"].replace(0, np.nan).interpolate().ffill().bfill()  # No zeros!
          # Avoid division by zero
        # Compute displacement
        df["dx"] = df["X"].diff().fillna(0)
        df["dy"] = df["Y"].diff().fillna(0)
        # Compute trajectory length
        df["trajectory_length"] = np.sqrt(df["dx"]**2 + df["dy"]**2)
        # Stroke speed
        stroke_speed = df["trajectory_length"].sum() / (df["Timestamp"].iloc[-1] - df["Timestamp"].iloc[0])
        # Compute velocity
        df["velocity"] = df["trajectory_length"] / df["dt"]
        df["velocity"] = df["velocity"].replace([np.inf, -np.inf], np.nan).interpolate().ffill().bfill()
        # Compute acceleration
        df["acceleration"] = df["velocity"].diff().fillna(0) / df["dt"]
        df["acceleration"] = df["acceleration"].replace([np.inf, -np.inf], np.nan).interpolate().ffill().bfill()
        # Compute jerk
        df["jerk"] = df["acceleration"].diff().fillna(0) / df["dt"]
        df["jerk"] = df["jerk"].replace([np.inf, -np.inf], np.nan).interpolate().ffill().bfill()
        # Number of changes in velocity and acceleration direction
        ncv = len(argrelextrema(df["velocity"].values, np.less)[0]) + len(argrelextrema(df["velocity"].values, np.greater)[0])
        nca = len(argrelextrema(df["acceleration"].values, np.less)[0]) + len(argrelextrema(df["acceleration"].values, np.greater)[0])
        # Path efficiency
        euclidean_distance = np.sqrt((df["X"].iloc[-1] - df["X"].iloc[0])**2 + (df["Y"].iloc[-1] - df["Y"].iloc[0])**2)
        path_efficiency = euclidean_distance / df["trajectory_length"].sum()
        # Stroke duration
        stroke_duration = df["Timestamp"].iloc[-1] - df["Timestamp"].iloc[0]
        # Pressure features
        mean_pressure = df["Pressure"].mean()
        std_pressure = df["Pressure"].std()
        num_pressure_drops = sum(df["Pressure"].diff().fillna(0) < -0.1)

        

        # Feature DataFrame
        feature_data = pd.DataFrame([[
            stroke_speed, df["velocity"].mean(), df["acceleration"].mean(), df["jerk"].mean(), 
            ncv, nca, path_efficiency, stroke_duration, mean_pressure, std_pressure, num_pressure_drops, label
        ]], columns=[
            "stroke_speed", "mean_velocity", "mean_acceleration", "mean_jerk", 
            "NCV", "NCA", "path_efficiency", "stroke_duration",
            "mean_pressure", "std_pressure", "num_pressure_drops", "label"
        ])

        dataframes_PaHaW.append(feature_data)

    # Combine all DataFrames
    if dataframes_PaHaW:
        return pd.concat(dataframes_PaHaW, ignore_index=True)
    else:
        return pd.DataFrame()


In [11]:
find_svc_files("/home/varsallz/Diagnostic_Aid_PD/data/Drawing/PaHaW/PaHaW/PaHaW_public/PD")

['/home/varsallz/Diagnostic_Aid_PD/data/Drawing/PaHaW/PaHaW/PaHaW_public/PD/00005/00005__1_1.svc',
 '/home/varsallz/Diagnostic_Aid_PD/data/Drawing/PaHaW/PaHaW/PaHaW_public/PD/00006/00006__1_1.svc',
 '/home/varsallz/Diagnostic_Aid_PD/data/Drawing/PaHaW/PaHaW/PaHaW_public/PD/00019/00019__1_1.svc',
 '/home/varsallz/Diagnostic_Aid_PD/data/Drawing/PaHaW/PaHaW/PaHaW_public/PD/00078/00078__1_1.svc',
 '/home/varsallz/Diagnostic_Aid_PD/data/Drawing/PaHaW/PaHaW/PaHaW_public/PD/00033/00033__1_1.svc',
 '/home/varsallz/Diagnostic_Aid_PD/data/Drawing/PaHaW/PaHaW/PaHaW_public/PD/00024/00024__1_1.svc',
 '/home/varsallz/Diagnostic_Aid_PD/data/Drawing/PaHaW/PaHaW/PaHaW_public/PD/00018/00018__1_1.svc',
 '/home/varsallz/Diagnostic_Aid_PD/data/Drawing/PaHaW/PaHaW/PaHaW_public/PD/00004/00004__1_1.svc',
 '/home/varsallz/Diagnostic_Aid_PD/data/Drawing/PaHaW/PaHaW/PaHaW_public/PD/00075/00075__1_1.svc',
 '/home/varsallz/Diagnostic_Aid_PD/data/Drawing/PaHaW/PaHaW/PaHaW_public/PD/00002/00002__1_1.svc',
 '/home/va

## Feature Extraction


In [12]:
# # Load UCI (Istanbul) dataset
# folder_path_HC = "/home/varsallz/Diagnostic_Aid_PD/data/Drawing/ISTANBUL/Improved Spiral Test Using Digitized Graphics Tablet for Monitoring Parkinson's Disease/data/Healthy"
# df_HC = feature_extraction_HandMotion(folder_path_HC, label=0)

# folder_path_PD = "/home/varsallz/Diagnostic_Aid_PD/data/Drawing/ISTANBUL/Improved Spiral Test Using Digitized Graphics Tablet for Monitoring Parkinson's Disease/data/PWP"
# df_PD = feature_extraction_HandMotion(folder_path_PD, label=1)

# df = pd.concat([df_PD, df_HC], axis=0, ignore_index=True)
# df = df.sample(frac=1).reset_index(drop=True)
# df = df.dropna()
# df.to_csv("HandMotionFeatures.csv", index=False)


# Load PaHaW dataset
HC1_folder_path = "/home/varsallz/Diagnostic_Aid_PD/data/Drawing/PaHaW/PaHaW/PaHaW_public/HC"
df_HC1 = feature_extraction_HandMotion_ext(HC1_folder_path, label=0)
df_HC1 = df_HC1.replace([np.inf, -np.inf], np.nan).ffill(axis=1)

PD1_folder_path = "/home/varsallz/Diagnostic_Aid_PD/data/Drawing/PaHaW/PaHaW/PaHaW_public/PD"
df_PD1= feature_extraction_HandMotion_ext(PD1_folder_path, label=1)
df_PD1 = df_PD1.replace([np.inf, -np.inf], np.nan).ffill(axis=1)

df1 = pd.concat([df_PD1, df_HC1], axis=0)
df1 = df1.sample(frac=1).reset_index(drop=True)

output_csv = "PaHaW_features_1.csv"
df1.to_csv(output_csv, index=False)

## Training

In [ ]:
# **Feature Scaling**
X = df1.drop(columns=["label"])
y = df1["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Standardize Data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply PCA
pca = PCA(n_components=0.95)  # Retain 95% variance
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Handle Class Imbalance with SMOTE 
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_pca, y_train)

#######################################################################################################

# **Model Training and Evaluation**
models = {
    "RandomForest": RandomForestClassifier(),
    "SVM": SVC(),
    "LogisticRegression": LogisticRegression(max_iter=500, solver = 'lbfgs'),
    "XGBoost": XGBClassifier()
}
param_grids = {
    "RandomForest": {
        "n_estimators": [100, 200, 300],
        "max_depth": [None, 10, 20],
        "min_samples_split": [2, 5, 10]
    },
    "SVM": {
        "C": [0.1, 1, 10],
        "kernel": ["linear", "rbf"]
    },
    "LogisticRegression": {
        "C": [0.01, 0.1, 1, 10]
    },
    "XGBoost": {
        "n_estimators": [100, 200, 300],
        "max_depth": [3, 5, 7],
        "learning_rate": [0.01, 0.1, 0.2]
    }
}
results = []
best_models = {}

for model_name, model in models.items():
    grid_search = GridSearchCV(model, param_grids[model_name], cv=5, scoring="accuracy", n_jobs=-1)
    grid_search.fit(X_train_balanced, y_train_balanced)
    best_models[model_name] = grid_search.best_estimator_
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")

    y_pred = best_models[model_name].predict(X_test_pca)
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    results.append([model_name, acc, precision, recall, f1])

    # Save trained models
    model_path = f"trained_models/{model_name}.pkl"
    os.makedirs("trained_models", exist_ok=True)
    pd.to_pickle(best_models[model_name], model_path)

        # Save Scaler & PCA
    os.makedirs("saved_models", exist_ok=True)
    with open("saved_models/scaler.pkl", "wb") as f:
        pickle.dump(scaler, f)

    with open("saved_models/pca.pkl", "wb") as f:
        pickle.dump(pca, f)


# **Save results**
results_df = pd.DataFrame(results, columns=["Model", "Accuracy", "Precision", "Recall", "F1-Score"])
results_df.to_csv("ModelPerformance_with_PaHaw.csv", index=False)

# Display results
print(results_df)


Best parameters for RandomForest: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 300}
Best parameters for SVM: {'C': 10, 'kernel': 'rbf'}
Best parameters for LogisticRegression: {'C': 1}
Best parameters for XGBoost: {'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200}
                Model  Accuracy  Precision    Recall  F1-Score
0        RandomForest  0.803571   0.791667  0.527778  0.633333
1                 SVM  0.696429   0.525000  0.583333  0.552632
2  LogisticRegression  0.638393   0.455446  0.638889  0.531792
3             XGBoost  0.834821   0.843137  0.597222  0.699187


In [ ]:
# # **Feature Scaling**
# X = df.drop(columns=["label"])
# y = df["label"]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# # Standardize Data
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # Apply PCA
# pca = PCA(n_components=0.95)  # Retain 95% variance
# X_train_pca = pca.fit_transform(X_train_scaled)
# X_test_pca = pca.transform(X_test_scaled)


# # Handle Class Imbalance with SMOTE 
# smote = SMOTE(random_state=42)
# X_train_balanced, y_train_balanced = smote.fit_resample(X_train_pca, y_train)

# #######################################################################################################

# # **Model Training and Evaluation**
# models = {
#     "RandomForest": RandomForestClassifier(),
#     "SVM": SVC(),
#     "LogisticRegression": LogisticRegression(max_iter=500, solver = 'lbfgs'),
#     "XGBoost": XGBClassifier()
# }
# param_grids = {
#     "RandomForest": {
#         "n_estimators": [100, 200, 300],
#         "max_depth": [None, 10, 20],
#         "min_samples_split": [2, 5, 10]
#     },
#     "SVM": {
#         "C": [0.1, 1, 10],
#         "kernel": ["linear", "rbf"]
#     },
#     "LogisticRegression": {
#         "C": [0.01, 0.1, 1, 10]
#     },
#     "XGBoost": {
#         "n_estimators": [100, 200, 300],
#         "max_depth": [3, 5, 7],
#         "learning_rate": [0.01, 0.1, 0.2]
#     }
# }
# results = []
# best_models = {}

# for model_name, model in models.items():
#     grid_search = GridSearchCV(model, param_grids[model_name], cv=5, scoring="accuracy", n_jobs=-1)
#     grid_search.fit(X_train_balanced, y_train_balanced)
#     best_models[model_name] = grid_search.best_estimator_
#     print(f"Best parameters for {model_name}: {grid_search.best_params_}")

#     # best_models[model_name].fit(X_train_balanced, y_train_balanced)
#     y_pred = best_models[model_name].predict(X_test_pca)

#     acc = accuracy_score(y_test, y_pred)
#     precision = precision_score(y_test, y_pred)
#     recall = recall_score(y_test, y_pred)
#     f1 = f1_score(y_test, y_pred)

#     results.append([model_name, acc, precision, recall, f1])

#     # Save trained models
#     model_path = f"trained_models/{model_name}.pkl"
#     os.makedirs("trained_models", exist_ok=True)
#     pd.to_pickle(best_models[model_name], model_path)

#     # Save Scaler & PCA
#     os.makedirs("saved_models", exist_ok=True)
#     with open("saved_models/scaler.pkl", "wb") as f:
#         pickle.dump(scaler, f)

#     with open("saved_models/pca.pkl", "wb") as f:
#         pickle.dump(pca, f)

# results_df = pd.DataFrame(results, columns=["Model", "Accuracy", "Precision", "Recall", "F1-Score"])
# results_df.to_csv("ModelPerformance_UCI.csv", index=False)
# print(results_df)

Best parameters for RandomForest: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 100}
Best parameters for SVM: {'C': 1, 'kernel': 'linear'}
Best parameters for LogisticRegression: {'C': 1}
Best parameters for XGBoost: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
                Model  Accuracy  Precision    Recall  F1-Score
0        RandomForest       0.8   0.833333  0.833333  0.833333
1                 SVM       1.0   1.000000  1.000000  1.000000
2  LogisticRegression       1.0   1.000000  1.000000  1.000000
3             XGBoost       1.0   1.000000  1.000000  1.000000


## Text on external dataset

In [99]:
# Load new dataset
new_df = pd.read_csv("HandMotionFeatures.csv")  # Replace with actual dataset path

if "label" in new_df.columns:
    X_new = new_df.drop(columns=["label"])
    y_new = new_df["label"]
else:
    X_new = new_df  # No labels, only features

# Load previously fitted scaler & PCA (assuming they were saved separately)
scaler = pd.read_pickle("saved_models/scaler.pkl")
pca = pd.read_pickle("saved_models/pca.pkl")

# Apply same preprocessing steps
X_new_scaled = scaler.transform(X_new)  
X_new_pca = pca.transform(X_new_scaled)  

# Load trained models
model_names = ["RandomForest", "SVM", "LogisticRegression", "XGBoost"]
models = {}

for name in model_names:
    model_path = f"trained_models/{name}.pkl"
    if os.path.exists(model_path):
        models[name] = pd.read_pickle(model_path)

# Evaluate models
if "label" in new_df.columns:  # If ground truth labels are available
    results = []

    for name, model in models.items():
        y_pred = model.predict(X_new_pca)
        acc = accuracy_score(y_new, y_pred)
        precision = precision_score(y_new, y_pred)
        recall = recall_score(y_new, y_pred)
        f1 = f1_score(y_new, y_pred)

        results.append([name, acc, precision, recall, f1])

    results_df = pd.DataFrame(results, columns=["Model", "Accuracy", "Precision", "Recall", "F1-Score"])
    print(results_df)
else:
    # Predict only (if no labels are available)
    predictions = {name: model.predict(X_new_pca) for name, model in models.items()}
    for name, pred in predictions.items():
        print(f"Predictions for {name}:", pred[:10])  # Show first 10 predictions


                Model  Accuracy  Precision    Recall  F1-Score
0        RandomForest  0.421053   0.545455  0.260870  0.352941
1                 SVM  0.684211   0.720000  0.782609  0.750000
2  LogisticRegression  0.289474   0.333333  0.173913  0.228571
3             XGBoost  0.421053   0.545455  0.260870  0.352941
